In [1]:
# Some las data are so large to read them and voxelize them.
# here I show a way to do it...which is memory frienldy.
# In 'pcd_read_write.ipynb' notebook, I showed it how to do it with pcd files
# but it did not work that way, since memory didn't matter at that notebook case.

In [2]:
# if case the large file is of format .laz:
# For large .laz files you need to install :
# >> conda install -c conda-forge lazrs-python 

In [3]:
import laspy
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
from pathlib import Path

In [5]:
file_addr = Path( "/home/sobhan/Downloads/lidar data/InterpineData/Plot1B_UNC_0_0.laz")

In [6]:
# file_addr = '/home/sobhan/Downloads/lidar data/InterpineData/LIR9_UNC_0_0 - Cloud-cleaned.las'

In [7]:
# laspy.read(file_addr)

In [8]:
las = laspy.read(str(file_addr))

In [9]:
las.header

<LasHeader(1.4, <PointFormat(1, 5 bytes of extra dims)>)>

In [10]:
las.header.point_format


<PointFormat(1, 5 bytes of extra dims)>

In [11]:

las.header.point_count



93516437

In [12]:
las.vlrs



[<ExtraBytesVlr(extra bytes structs: 2)>, <laspy.vlrs.known.WktCoordinateSystemVlr object at 0x7fd9eb385070>]

In [13]:
# since the code format is 1.1 it is categorized as sth that must be read from
# https://pro.arcgis.com/en/pro-app/3.0/help/data/las-dataset/storing-lidar-data.htm



list(las.point_format.dimension_names)



['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'gps_time',
 'Ring',
 'Range']

In [14]:
las.X#, las.Y,las.Z

array([ 946,  774,  706, ..., -224, -277, -322], dtype=int32)

In [15]:

# 1 = Unassigned
# 2 = Ground
# 5 = High Vegetation
# 6 = Building
las.classification

<SubFieldView([0 0 0 ... 0 0 0])>

In [16]:
las.X.shape

(93516437,)

In [17]:
las.X.max(), las.X.min()

(15073, -14925)

In [18]:
coef_cubicize = (las.X.max()-las.X.min())/255
coef_cubicize

117.63921568627451

In [19]:
(las.X-las.X.min())/255

array([62.23921569, 61.56470588, 61.29803922, ..., 57.65098039,
       57.44313725, 57.26666667])

In [20]:
# def slotting(indices, cube_size):
#     '''quite like cube_voxel(indices, cube_size) function in ./pcd_read_write.ipynb  notebook
#     but this one is only works on on 1d np.ndarray not  a 3d np.ndarray. You can also use that function
#     but here just a verion of that...
#     '''
#     cube_size = cube_size-1 #(ex. 256-->max will be 255)
#     coef_cubicize = (indices.max(axis=0)-indices.min(axis=0))/cube_size

#     return ((indices-indices.min(axis=0))//coef_cubicize).astype(int)
# #     cube_voxels = cube_voxels.astype(int)
# #     return cube_voxels

In [21]:
# slotting(las.X, 256)

array([134, 133, 132, ..., 124, 124, 124])

In [22]:

# If I use 3d ndarry I would run out of mem, so I use for loop for values at the cost of losing time
# but saving mem, Since .las file here is supposed to be really big....

def cube_las(las, cube_size):
    '''a slower voxelizing for las large files but yet using less mem. 
    build_cube() function in utils.py  is doing 
    the same but uses more mem...So for large las files I use this function'''
    vox = np.zeros((cube_size,cube_size,cube_size))
    coef_x = (las.X.max()-las.X.min())/(cube_size-1)
    min_x  = las.X.min()

    coef_y = (las.Y.max()-las.Y.min())/(cube_size-1)
    min_y  = las.Y.min()

    coef_z = (las.Z.max()-las.Z.min())/(cube_size-1)
    min_z  = las.Z.min()

    for index in range(las.X.shape[0]):
        ix = ((las.X[index]-min_x)//coef_x).astype(int)
        iy = ((las.Y[index]-min_y)//coef_y).astype(int)
        iz = ((las.Z[index]-min_z)//coef_z).astype(int)
        vox[ix, iy, iz] = 1
    
    return vox     
    #     pass

In [24]:
# I use 360 instead of 256, since I may loose some info after cutting and cleaning and if 
# I select 256, and then clean to 245, to say,  enlarging from 245 to 256 leave me with some 
# blanks in the shapes... So I keep it 360 to never have such trouble for enlarge...sicne
# I won't have such trouble from 345 to 256 resizing....

vox = cube_las(las, cube_size=360)

In [28]:
vox.shape

(360, 360, 360)

In [29]:
from utils import save_mat, save_xyz

In [30]:
save_xyz(vox, 'output-complete-cleaned-Brendan.xyz')

voxel cube data is stored at  output-complete-cleaned-Brendan.xyz


In [ ]:
save_mat(vox, 'output-complete-cleaned.mat')

In [ ]:
# above code might be double and you need to convert it to int
# in following junks you can convert it to int....Just in case

In [ ]:
import scipy.io
mat = scipy.io.loadmat('output-complete.mat')

In [ ]:
vox = mat['vox'].astype(int)

In [ ]:
save_mat(vox, 'output-complete1.mat')

In [ ]:
save_xyz(vox, 'output-complete1.xyz')